<a href="https://colab.research.google.com/github/kavyajeetbora/PSENet.pytorch/blob/master/PSENet_trial_run.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import matplotlib.pyplot as plt
import numpy as np

import torch
from torch import nn
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset

In [2]:
!pip install pyclipper

     |████████████████████████████████| 133kB 6.5MB/s 


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [4]:
import shutil, os
os.chdir('/content')
directory = '/content/cloned-repo'
if os.path.exists(directory):
  shutil.rmtree(directory)

!git clone https://github.com/kavyajeetbora/PSENet.pytorch.git /content/cloned-repo
print("Cloned the repository")
os.chdir('/content/cloned-repo')
!ls

Cloning into '/content/cloned-repo'...
remote: Enumerating objects: 82, done.
remote: Counting objects: 100% (82/82), done.
remote: Compressing objects: 100% (75/75), done.
remote: Total 378 (delta 44), reused 15 (delta 7), pack-reused 296
Receiving objects: 100% (378/378), 8.13 MiB | 9.83 MiB/s, done.
Resolving deltas: 100% (170/170), done.
Cloned the repository
cal_recall  imgs		     predict.py		     README.md
config.py   install_dependencies.sh  pse		     train.py
dataset     LICENSE		     PSENet.ipynb	     utils
eval.py     models		     PSENet_trial_run.ipynb


In [5]:
## unzipping the files
from zipfile import ZipFile

def unzip_files(file,output_dir):
  with ZipFile(file, 'r') as zipObj:
    # Extract all the contents of zip file in current directory
    zipObj.extractall(output_dir)
  print('Extracted',len(output_dir),'to output dir')

image_files = '/content/drive/My Drive/Colab Notebooks/padh.ai.notebooks/15. Object Detection/AI4Bharat Dataset/Training Set/Images/Images.Zip'
annotation_files = '/content/drive/My Drive/Colab Notebooks/padh.ai.notebooks/15. Object Detection/AI4Bharat Dataset/Training Set/Annotations/Annotations.Zip'

directory = '/content/Train Set'
if os.path.isdir(directory):
  shutil.rmtree(directory)
 
os.mkdir('Train Set')
print('Created a new directory')

Created a new directory


In [6]:
%%time
unzip_files(image_files, output_dir='Train Set/Images')

Extracted 16 to output dir
CPU times: user 1.27 s, sys: 427 ms, total: 1.69 s
Wall time: 5.48 s


In [7]:
%%time
unzip_files(annotation_files, output_dir='Train Set/Annotations')

Extracted 21 to output dir
CPU times: user 476 ms, sys: 214 ms, total: 689 ms
Wall time: 2.03 s


In [8]:
print(len(os.listdir('Train Set/Images')))

5000


In [9]:
print(len(os.listdir('Train Set/Annotations')))

5000


In [0]:
from dataset.data_utils import *
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset

In [11]:
data_dir  = 'Train Set'
len(os.listdir(data_dir+ '/Images'))

5000

In [12]:
os.listdir(data_dir)

['Images', 'Annotations']

In [13]:
train_data = MyDataset('Train Set',transform=transforms.ToTensor())
len(train_data)

5000

In [0]:
device = torch.device("cuda:0")

In [15]:
%%time
train_loader = DataLoader(dataset=train_data, batch_size=16, shuffle=True)
img, labels, training_mask = next(iter(train_loader))
print(img.size(), labels.size(), training_mask.size())

torch.Size([16, 3, 640, 640]) torch.Size([16, 6, 640, 640]) torch.Size([16, 640, 640])
CPU times: user 1.1 s, sys: 271 ms, total: 1.37 s
Wall time: 1.06 s


In [0]:
img, labels, training_mask = img.to(device), labels.to(device), training_mask.to(device)

In [17]:
from models import PSENet
from models.loss import PSELoss
import torch

model = PSENet(backbone='resnet18', pretrained=True, result_num=6, scale=0.5)
model.to(device)
criterion = PSELoss(Lambda=0.7, ratio=3, reduction='mean')
optimizer = torch.optim.Adam(model.parameters(), lr=2e-3)

Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /root/.cache/torch/checkpoints/resnet18-5c106cde.pth
100%|██████████| 44.7M/44.7M [00:01<00:00, 26.6MB/s]


In [18]:
y1 = model(img)
loss_c, loss_s, loss = criterion(y1, labels, training_mask)

print(y1.size())
print(loss_c, loss_s, loss)

torch.Size([16, 6, 640, 640])
tensor(0.6442, device='cuda:0', grad_fn=<MeanBackward0>) tensor(0.9198, device='cuda:0', grad_fn=<MeanBackward0>) tensor(0.7269, device='cuda:0', grad_fn=<AddBackward0>)


In [0]:
# !chmod +x install_dependencies.sh # make shell script executable
# !./install_dependencies.sh # run the shell script